<a href="https://colab.research.google.com/github/lorenafc/Useful_Knowledge/blob/main/fix_city_names_encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import libraries
import pandas as pd
import html  # Import the html module for decoding HTML entities

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# upload the file
from google.colab import files
uploaded=files.upload()

Saving additional_authors.xlsx to additional_authors.xlsx


In [2]:
file_name = 'additional_authors.xlsx'

# Read the Excel file
authors_small_cities = pd.read_excel(file_name, engine='openpyxl')

authors_small_cities_copy = authors_small_cities.copy()

In [16]:
authors_small_cities.head()

,starturl,originalbirth,originaldeath,birthyear,deathyear,nameandbirthdeathyear,georeferenceurl,death,born,active,borncity,deathcity,activecity
0,http://viaf.org/viaf/81048146,1570.0,1634.0,1570,1634.0,"Carneiro, Tomé Tavares 1570-1634",viafprocessedfile,NaN,,Rio Tinto,,NaN,Rio Tinto
1,http://viaf.org/viaf/180581759,0.0,1775.0,NaN,1775.0,"Kolíček, Caesarij, -1775",viafprocessedfile,Cieszin,,NaN,,Cieszin,NaN
2,http://viaf.org/viaf/100161519,0.0,1570.0,NaN,1570.0,"Hertel, Jakob 15..?-1570",http://catalogue.bnf.fr/ark:/12148/cb15011774s,NaN,(Chur,NaN,(Chur,NaN,NaN
3,http://viaf.org/viaf/169350346,1699.0,1770.0,1699,1770.0,"Ferro Caaveyro, Lucas Antonio ca. 1699-1770",www.wikidata.org/wiki/Q3393623#sitelinks-wikip...,NaN,A Capela,NaN,A Capela,NaN,NaN
4,http://viaf.org/viaf/67541728,1502.0,1600.0,1502,1600.0,"Aparicio, Sebastián de 1502-1600 Santo",www.wikidata.org/wiki/Q2486114#sitelinks-wikip...,Puebla City,A GudiÃ±a,NaN,A GudiÃ±a,Puebla City,NaN


#ENCODING FIX

(not all of the names were fixed)

In [17]:
#source: chatGPT and Lucas Koren

def setup_and_fix_encoding(df):
    """
    Creates necessary city columns if they don't exist and applies encoding fixes to city name columns.

    :param df: pd.DataFrame, the DataFrame containing the original city data
    :return: pd.DataFrame, the DataFrame with fixed encodings for city names
    """
    # Define the mappings from original columns to new city name columns if they don't exist
    column_mappings = {
        'born': 'borncity',
        'death': 'deathcity',
        'active': 'activecity'
    }

    # Create new city columns from existing data if they don't exist
    for original_col, new_col in column_mappings.items():
        if new_col not in df.columns:
            df[new_col] = df[original_col]

    # Define possible encodings
    encodings = ['utf-8', 'latin1', 'windows-1252', 'windows-1250', "latin2", "iso-8859-1", "iso-8859-2", 'iso-8859-15',
                 'macroman', 'ascii', 'cp1250', 'cp1251', 'cp1253', 'cp1254', 'cp1257', 'iso-8859-3', 'iso-8859-4',
                 'iso-8859-5', 'iso-8859-6', 'iso-8859-7', 'iso-8859-8', 'iso-8859-9', 'koi8-r', 'koi8-u']

    def fix_encoding(text):
        if not isinstance(text, str):
            return text
        # Decode HTML entities first
        text = html.unescape(text)
        # Try decoding and encoding combinations
        for enc1 in encodings:
            for enc2 in encodings:
                if enc1 != enc2:
                    try:
                        decoded_text = text.encode(enc2, errors='replace').decode(enc1, errors='replace')
                        if decoded_text.encode(enc1, errors='replace').decode(enc2, errors='replace') == text:
                            return decoded_text
                    except (UnicodeEncodeError, UnicodeDecodeError):
                        continue

        return text

    # Apply the encoding fix function to the newly created city name columns
    for _, new_col in column_mappings.items():
        df[new_col] = df[new_col].apply(fix_encoding)

    return df



In [18]:
authors_small_cities_fixed_encoding = setup_and_fix_encoding(authors_small_cities)
print(authors_small_cities_fixed_encoding.head())

                         starturl  originalbirth  originaldeath birthyear  \
0   http://viaf.org/viaf/81048146         1570.0         1634.0      1570   
1  http://viaf.org/viaf/180581759            0.0         1775.0       NaN   
2  http://viaf.org/viaf/100161519            0.0         1570.0       NaN   
3  http://viaf.org/viaf/169350346         1699.0         1770.0      1699   
4   http://viaf.org/viaf/67541728         1502.0         1600.0      1502   

   deathyear                        nameandbirthdeathyear  \
0     1634.0             Carneiro, Tomé Tavares 1570-1634   
1     1775.0                     Kolíček, Caesarij, -1775   
2     1570.0                     Hertel, Jakob 15..?-1570   
3     1770.0  Ferro Caaveyro, Lucas Antonio ca. 1699-1770   
4     1600.0       Aparicio, Sebastián de 1502-1600 Santo   

                                     georeferenceurl        death       born  \
0                                  viafprocessedfile          NaN              
1         

In [19]:
def clean_and_rename_dataset(df):
    """
    Renames columns in the DataFrame, drops unnecessary columns, and reorders the remaining columns.

    :param df: pd.DataFrame, the original DataFrame to be cleaned and renamed
    :return: pd.DataFrame, the cleaned and reordered DataFrame
    """
    # Rename the columns
    df.rename(columns={'cleanedbirth': 'birthyear',
                       'cleaneddeath': 'deathyear',
                       'finalname': 'nameandbirthdeathyear'},
              inplace=True)

    # Drop unnecessary columns
    df_cleaned = df.drop(columns=["originalbirth", "originaldeath"])

    # Reorder the columns
    df_cleaned = df_cleaned[['starturl', 'birthyear', 'deathyear', 'nameandbirthdeathyear',
                             'georeferenceurl', 'born', 'borncity', 'death', # I kept the original columns with the city names for comparison
                             'deathcity', 'active', 'activecity']]

    return df_cleaned


In [20]:
authors_small_cities_fixed_encoding_cleaned = clean_and_rename_dataset(authors_small_cities_fixed_encoding)
print(authors_small_cities_fixed_encoding_cleaned.head(16))

                          starturl birthyear  deathyear  \
0    http://viaf.org/viaf/81048146      1570     1634.0   
1   http://viaf.org/viaf/180581759       NaN     1775.0   
2   http://viaf.org/viaf/100161519       NaN     1570.0   
3   http://viaf.org/viaf/169350346      1699     1770.0   
4    http://viaf.org/viaf/67541728      1502     1600.0   
5   http://viaf.org/viaf/121452165      1320     1394.0   
6    http://viaf.org/viaf/62383957      1538     1592.0   
7    http://viaf.org/viaf/67541507      1530     1596.0   
8    http://viaf.org/viaf/78669554      1640     1705.0   
9    http://viaf.org/viaf/79445202      1731     1797.0   
10   http://viaf.org/viaf/24946396      1706     1761.0   
11   http://viaf.org/viaf/59506396      1663     1717.0   
12   http://viaf.org/viaf/84185160      1716     1761.0   
13   http://viaf.org/viaf/59517896       NaN     1604.0   
14   http://viaf.org/viaf/29054119      1731     1776.0   
15   http://viaf.org/viaf/72325500      1659     1728.0 

In [21]:
#Saving excel format
file_name = 'additional_authors_fixed_encoding.xlsx'
authors_small_cities_fixed_encoding_cleaned.to_excel(file_name)

In [22]:
#Saving csv format
file_name = 'additional_authors_fixed_encoding.csv'
authors_small_cities_fixed_encoding_cleaned.to_csv(file_name)